# CAGRA Demo with NVIDIA RAFT

Learn more about CAGRA [here](https://arxiv.org/pdf/2308.15136)!

In [1]:
!nvidia-smi

Mon Aug  5 14:52:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Check which version of CUDA you are using, if 11.x -- you will need to use `pylibraft-cu11`.

In [ ]:
!pip install pylibraft-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install raft-dask-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cupy==13.2.0
!pip install sentence-transformers==3.0.1

# Dataset

In [4]:
!wget "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip"

--2024-08-05 15:01:38--  https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2448432 (2.3M) [application/zip]
Saving to: ‘nfcorpus.zip’

nfcorpus.zip        100%[===================>]   2.33M  2.12MB/s    in 1.1s    

2024-08-05 15:01:41 (2.12 MB/s) - ‘nfcorpus.zip’ saved [2448432/2448432]



In [5]:
!unzip nfcorpus.zip

Archive:  nfcorpus.zip
   creating: nfcorpus/
   creating: nfcorpus/qrels/
  inflating: nfcorpus/qrels/train.tsv  
  inflating: nfcorpus/qrels/test.tsv  
  inflating: nfcorpus/qrels/dev.tsv  
  inflating: nfcorpus/corpus.jsonl   
  inflating: nfcorpus/queries.jsonl  


In [6]:
import json

dataset = "nfcorpus"

with open(f"./{dataset}/corpus.jsonl", "r") as json_file:
  json_list = list(json_file)

error_counter = 0

corpus = []
org_docID_to_seq_docID = {} # We use this key-value to match the relevance labels for query-doc pairs

for idx in range(0, len(json_list), 1):
  result = json.loads(json_list[idx])
  new_doc_obj = {}
  new_doc_obj["document"] = result["text"]
  # This might give you trouble, some BEIR datasets are string keys, others int!!
  org_docID_to_seq_docID[(result["_id"])] = idx
  new_doc_obj["DocID"] = idx

  corpus.append(new_doc_obj)

print(len(corpus))

3633


In [7]:
import json

with open(f"./{dataset}/queries.jsonl", "r") as json_file:
  json_list = list(json_file)

queries = []

for json_str in json_list:
  result = json.loads(json_str)
  new_query_obj = {}
  new_query_obj["queryID"] = result["_id"] # NOTE some are string keys others int
  new_query_obj["query"] = result["text"]

  queries.append(new_query_obj)

print(len(queries))

3237


In [8]:
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer

model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L12-v2")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L12-v2")
model.save_pretrained("all-MiniLM-L12-v2")
tokenizer.save_pretrained("all-MiniLM-L12-v2")

model = SentenceTransformer("all-MiniLM-L12-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
import cupy as cp

corpus_embeddings = []
for item in corpus:
  doc_embedding = model.encode(item["document"])
  corpus_embeddings.append(doc_embedding)

cp_corpus_embeddings = cp.asarray(corpus_embeddings)

In [10]:
from pylibraft.neighbors import cagra

In [11]:
%%time
params = cagra.IndexParams(
    intermediate_graph_degree=32,
    graph_degree=16,
    build_algo="nn_descent"
)
cagra_index = cagra.build(params, cp_corpus_embeddings)
search_params = cagra.SearchParams(algo="multi_cta")

CPU times: user 258 ms, sys: 69 ms, total: 327 ms
Wall time: 333 ms


In [51]:
import time
import torch

def search_raft_cagra(query, top_k = 5):
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = model.encode(query, convert_to_tensor=True)

    start_time = time.time()
    hits = cagra.search(search_params, cagra_index, question_embedding[None], top_k)
    end_time = time.time()

    # Output of top-k hits
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    print("Input question:", query)

    score_tensor = torch.as_tensor(hits[0], device='cpu')
    index_tensor = torch.as_tensor(hits[1], device='cpu')

    print("CAGRA Search Results: \n")
    for k in range(top_k):
      print("\t{:.3f}\t{}".format(score_tensor[0, k], corpus[index_tensor[0, k]]["document"]))

In [52]:
%%time
search_raft_cagra(query="What does taking supplemental B12 vitamins help with?")

Results (after 0.001 seconds):
Input question: What does taking supplemental B12 vitamins help with?
CAGRA Search Results: 

	20.404	Vitamin B12 deficiency anemia may have psychiatric manifestations preceding the hematological symptoms. Although a variety of symptoms are described, there are only sparse data on the role of vitamin B12 in depression. We report a case of vitamin B12 deficiency presenting with recurrent episodes of depression.
	20.468	Elevated total plasma homocysteine has been linked to the development of cognitive impairment and dementia in later life and this can be reliably lowered by the daily supplementation of vitamin B6, B12, and folic acid. We performed a systematic review and meta-analysis of 19 English language randomized, placebo-controlled trials of homocysteine lowering B-vitamin supplementation of individuals with and without cognitive impairment at the time of study entry. We standardized scores to facilitate comparison between studies and to enable us to 